In [ ]:
!pip install rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 61.4 MB/s eta 0:00:00


In [ ]:
import geopandas as gpd
import rasterio
from rasterio.mask import mask
from rasterio.warp import reproject, Resampling
import json
from shapely.geometry import Polygon, Point
import numpy as np
import matplotlib.pyplot as plt
from rasterio.transform import xy
import folium

# === STEP 1: Load Zoning Data ===
zones = gpd.read_file("HomogenousSubZones/HomogenousSubZones.shp")
zones["Max Height"] = 10.5  # Or use a CSV for per-zone rules

# === STEP 2: Load DSM and DTM ===
dsm = rasterio.open("P5_PAN_CD_N30_000_E076_000_DEM_30m.tif")
dtm = rasterio.open("n30_e076_1arc_v3.tif")

# Match CRS
zones = zones.to_crs(dsm.crs)

# === STEP 3: Clip DSM/DTM to Zones ===
dsm_clip, dsm_transform = mask(dsm, zones.geometry, crop=True)
dsm_array = dsm_clip[0].astype(float)

dtm_clip, _ = mask(dtm, zones.geometry, crop=True)
dtm_array = dtm_clip[0].astype(float)

# === STEP 4: Resample DTM to DSM ===
resampled_dtm = np.empty_like(dsm_array)
reproject(
    dtm_array, resampled_dtm,
    src_transform=dtm.transform,
    src_crs=dtm.crs,
    dst_transform=dsm_transform,
    dst_crs=dsm.crs,
    dst_resolution=dsm.res,
    resampling=Resampling.bilinear
)

# === STEP 5: Compute Height Map ===
height_map = dsm_array - resampled_dtm
height_map[height_map < 0] = 0

# === STEP 6: Load OSM Buildings from export.json ===
with open("export.json") as f:
    data = json.load(f)

# Reconstruct geometries
features = []
node_map = {el["id"]: (el["lon"], el["lat"]) for el in data["elements"] if el["type"] == "node"}

for el in data["elements"]:
    if el["type"] == "way" and "building" in el.get("tags", {}):
        nodes = el["nodes"]
        coords = [node_map[nid] for nid in nodes if nid in node_map]
        if len(coords) >= 3:
            geom = Polygon(coords)
            tags = el["tags"]
            levels = tags.get("building:levels")
            height = tags.get("height")
            est_height = float(height) if height else (int(levels) * 3 if levels else None)
            features.append({
                "geometry": geom,
                "height": est_height,
                "name": tags.get("name", ""),
                "levels": levels
            })

buildings_gdf = gpd.GeoDataFrame(features, crs="EPSG:4326")
buildings_gdf = buildings_gdf.to_crs(dsm.crs)

# === STEP 7: Check Violations ===
buildings_within = gpd.sjoin(buildings_gdf, zones, predicate="within")
buildings_within["Max Height"] = 10.5  # Or use value from zone
buildings_within["violation"] = buildings_within["height"] > buildings_within["Max Height"]

# === STEP 8: Map Violations ===
m = folium.Map(location=[30.741, 76.768], zoom_start=14)

for _, row in buildings_within.iterrows():
    if not row["violation"]:
        continue
    centroid = row.geometry.centroid
    folium.CircleMarker(
        location=[centroid.y, centroid.x],
        radius=5,
        color='red',
        fill=True,
        popup=f"{row.get('name', 'Unnamed')}, H={row['height']}m"
    ).add_to(m)

m.save("osm_violation_map.html")
print("Map saved as osm_violation_map.html")


Map saved as osm_violation_map.html


In [ ]:
!pip install pyrosm

  Using cached pyrosm-0.6.2.tar.gz (2.5 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 kB 1.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pyrobuf-0.9.3-cp312-cp312-linux_x86_64.whl
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 24.2 MB/s eta 0:00:00
  Created wheel for pyrosm: filename=pyrosm-0.6.2-cp312-cp312-linux_x86_64.whl size=8177432 sha256=e0ae5d332b70883a1ae4f7e3e16777ada646554276e70b395de52efe1047da26
  Stored in directory: /root/.cache/pip/wheels/33/aa/81/7751cbc2166a4d8dfd6da1cd0c44979037568083df9b065bc8
  Created wheel for cykhash: filename=cykhash-2.0.1-cp312-cp312-linux_x86_64.whl size=3938561 sha256=a7f0401cb78289dbbc0aa2863fe043d3303e2f547f4c497c44640b3cf8dd100b
  Stored in directory: /root/.cach

In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
import rasterio
from rasterio.mask import mask
from rasterio.warp import reproject, Resampling
from shapely.geometry import Polygon
import folium
import re
from pyrosm import OSM

# -------------------
# Helper functions
# -------------------
def extract_number(value):
    """Extract first number from a string (e.g., '15 meters' -> 15.0)."""
    if pd.isna(value):
        return None
    nums = re.findall(r"[\d\.]+", str(value))
    return float(nums[0]) if nums else None

# -------------------
# Step 1: Load policy
# -------------------
print("Reading policy CSV...")
policy = pd.read_csv("policy.csv")
policy = policy.set_index(policy.columns[0])  # "Parameter" column becomes index

# Extract height + setback rules
height_limits = {col: extract_number(policy.loc["Total Height Limit", col]) for col in policy.columns}
setback_limits = {col: extract_number(policy.loc["Front Setback", col]) for col in policy.columns}
print("Available land-use categories in policy:", list(policy.columns))

# -------------------
# Step 2: Load DSM & DTM
# -------------------
print("Opening rasters...")
dsm = rasterio.open("P5_PAN_CD_N30_000_E076_000_DEM_30m.tif")
dtm = rasterio.open("n30_e076_1arc_v3.tif")

# Clip to Chandigarh boundary (optional if large rasters)
# For now just read full
dsm_array = dsm.read(1).astype(float)
dtm_array = dtm.read(1).astype(float)

# Resample DTM to DSM grid
print("Resampling DTM to DSM grid...")
resampled_dtm = np.empty_like(dsm_array)
reproject(
    dtm_array, resampled_dtm,
    src_transform=dtm.transform,
    src_crs=dtm.crs,
    dst_transform=dsm.transform,
    dst_crs=dsm.crs,
    dst_resolution=dsm.res,
    resampling=Resampling.bilinear
)

# Height map
height_map = dsm_array - resampled_dtm
height_map[height_map < 0] = 0

# -------------------
# Step 3: Load OSM (buildings + landuse)
# -------------------
print("Loading OSM data...")
osm = OSM("chandigarh.pbf")

# Buildings
buildings_gdf = osm.get_data_by_custom_criteria(
    custom_filter={"building": True},
    filter_type="keep",
    keep_nodes=False,
    keep_relations=False
)

# Landuse polygons
landuse_gdf = osm.get_data_by_custom_criteria(
    custom_filter={"landuse": True},
    filter_type="keep",
    keep_nodes=False,
    keep_relations=False
)

# Match CRS with rasters
buildings_gdf = buildings_gdf.to_crs(dsm.crs)
landuse_gdf = landuse_gdf.to_crs(dsm.crs)

# -------------------
# Step 4: Assign policy categories
# -------------------
def map_landuse_to_policy(osm_tag):
    if osm_tag in ["residential"]:
        return "Residential Housing (≤ 250 sq. yd.)"
    if osm_tag in ["industrial"]:
        return "Industrial Buildings"
    if osm_tag in ["commercial", "retail"]:
        return "Mixed-Use Commercial"
    # fallback
    return "Residential Housing (≤ 250 sq. yd.)"

landuse_gdf["PolicyClass"] = landuse_gdf["landuse"].apply(map_landuse_to_policy)

# -------------------
# Step 5: Spatial join buildings with landuse
# -------------------
buildings_within = gpd.sjoin(buildings_gdf, landuse_gdf[["geometry","PolicyClass"]], predicate="within")

# Attach rules
buildings_within["MaxHeight"] = buildings_within["PolicyClass"].map(height_limits)
buildings_within["FrontSetback"] = buildings_within["PolicyClass"].map(setback_limits)

# Use building height tag if available, otherwise sample from DSM/DTM
def estimate_height(row):
    if "height" in row and pd.notna(row["height"]):
        try:
            return float(row["height"])
        except:
            return None
    if "building:levels" in row and pd.notna(row["building:levels"]):
        try:
            return float(row["building:levels"]) * 3  # assume 3m/level
        except:
            return None
    return None

buildings_within["EstHeight"] = buildings_within.apply(estimate_height, axis=1)

# Fallback: sample DSM-DTM height at centroid
rows, cols = rasterio.transform.rowcol(dsm.transform,
                                       buildings_within.geometry.centroid.x,
                                       buildings_within.geometry.centroid.y)
sampled_heights = height_map[rows, cols]
buildings_within["EstHeight"] = buildings_within["EstHeight"].fillna(pd.Series(sampled_heights))

# -------------------
# Step 6: Check violations
# -------------------
buildings_within["HeightViolation"] = buildings_within["EstHeight"] > buildings_within["MaxHeight"]

# Boundary violation: check if building footprint crosses outside inward-buffered landuse polygon
boundary_violations = []
for idx, row in buildings_within.iterrows():
    try:
        zone_geom = landuse_gdf.loc[row["index_right"]].geometry
        buffer_geom = zone_geom.buffer(-row["FrontSetback"]) if row["FrontSetback"] else zone_geom
        boundary_violations.append(not buffer_geom.contains(row.geometry))
    except:
        boundary_violations.append(False)

buildings_within["BoundaryViolation"] = boundary_violations

# -------------------
# Step 7: Map Violations
# -------------------
print("Creating map...")
m = folium.Map(location=[30.741, 76.768], zoom_start=13, tiles="OpenStreetMap")

for _, row in buildings_within.iterrows():
    centroid = row.geometry.centroid
    if row["HeightViolation"] or row["BoundaryViolation"]:
        color = "red" if row["HeightViolation"] else "orange"
        folium.CircleMarker(
            location=[centroid.y, centroid.x],
            radius=5,
            color=color,
            fill=True,
            popup=f"Use={row['PolicyClass']}, H={row['EstHeight']}m, Max={row['MaxHeight']}m"
        ).add_to(m)

m.save("violation_map.html")
print("Map saved as violation_map.html")


Reading policy CSV...
Available land-use categories in policy: ['Residential Housing (≤ 250 sq. yd.)', 'Residential Housing (251 – 500 sq. yd.)', 'Residential Housing (> 500 sq. yd.)', 'Group Housing', 'Apartment Complexes', 'Mixed-Use Zoning', 'Office Buildings', 'Shopping Complexes', 'Hotels & Lodging', 'Industrial Buildings', 'Mixed-Use Commercial']
Opening rasters...
Resampling DTM to DSM grid...
Loading OSM data...
Creating map...
✅ Map saved as violation_map.html


In [ ]:
import folium
from folium.features import GeoJsonTooltip
import geopandas as gpd

# Ensure AreaViolation column exists
buildings_within["AreaViolation"] = buildings_within.apply(check_area_violation, axis=1)

# Convert GeoDataFrame to GeoJSON
geojson_data = buildings_within.to_crs(epsg=4326).to_json()  # folium requires WGS84 (lat/lon)

# Map center
map_center = [buildings_within.geometry.centroid.y.mean(),
              buildings_within.geometry.centroid.x.mean()]

m = folium.Map(location=map_center, zoom_start=16, tiles="CartoDB positron")

# Style function
def style_function(feature):
    # Use get to avoid KeyError
    violation = feature['properties'].get('AreaViolation', False)
    if violation:
        return {'fillColor': 'red', 'color': 'red', 'weight': 2, 'fillOpacity': 0.6}
    else:
        return {'fillColor': 'green', 'color': 'green', 'weight': 2, 'fillOpacity': 0.4}

# Add GeoJSON layer
folium.GeoJson(
    geojson_data,
    style_function=style_function,
    tooltip=GeoJsonTooltip(
        fields=["PolicyClass", "Area_sqyd", "AreaViolation"],
        aliases=["Policy Class:", "Area (sq.yd):", "Violation:"],
        localize=True
    )
).add_to(m)

# Save map
m.save("building_area_violation_map.html")


In [ ]:
import geopandas as gpd
import folium
from folium.features import GeoJsonTooltip

# ---- 1. Compute building footprint area ----
# Assuming buildings_within is your GeoDataFrame
buildings_within["Area_sqm"] = buildings_within.geometry.area
buildings_within["Area_sqyd"] = buildings_within["Area_sqm"] / 0.836127  # 1 sq yd = 0.836127 sqm

# ---- 2. Define function to check violations ----
def check_area_violation(row):
    if "≤ 250" in row["PolicyClass"]:
        return row["Area_sqyd"] > 250
    elif "251 – 500" in row["PolicyClass"]:
        return not (251 <= row["Area_sqyd"] <= 500)
    elif "> 500" in row["PolicyClass"]:
        return row["Area_sqyd"] <= 500
    else:
        return False

# ---- 3. Apply function ----
buildings_within["AreaViolation"] = buildings_within.apply(check_area_violation, axis=1)

# ---- 4. Convert to GeoJSON for folium ----
geojson_data = buildings_within.to_crs(epsg=4326).to_json()  # folium requires WGS84 (lat/lon)

# ---- 5. Map center ----
map_center = [
    buildings_within.geometry.centroid.y.mean(),
    buildings_within.geometry.centroid.x.mean()
]

# ---- 6. Create folium map ----
m = folium.Map(location=map_center, zoom_start=16, tiles="CartoDB positron")

# ---- 7. Style function to highlight only violations ----
def style_function(feature):
    violation = feature['properties'].get('AreaViolation', False)
    if violation:
        return {'fillColor': 'red', 'color': 'red', 'weight': 2, 'fillOpacity': 0.6}
    else:
        return {'fillColor': 'transparent', 'color': 'gray', 'weight': 1, 'fillOpacity': 0}  # Optional: hide non-violations

# ---- 8. Add GeoJSON layer ----
folium.GeoJson(
    geojson_data,
    style_function=style_function,
    tooltip=GeoJsonTooltip(
        fields=["PolicyClass", "Area_sqyd", "AreaViolation"],
        aliases=["Policy Class:", "Area (sq.yd):", "Violation:"],
        localize=True
    )
).add_to(m)

# ---- 9. Save map ----
m.save("building_area_violation_map.html")

print("Map saved as building_area_violation_map.html")


Map saved as building_area_violation_map.html
